<a href="https://colab.research.google.com/github/AYUSH676-JARVIS/Safe-Crowd-AI-powered-crowd-management-and-stampede-prevention-system./blob/main/Latent_Semantic_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [5]:
pip install nltk

In [6]:
import nltk

In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

# Ensure NLTK data is downloaded (run once)
import nltk
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True) # Explicitly download punkt_tab

# Sample DataFrame
data = {
    'document': [
        'The quick brown fox jumps over the lazy dog.',
        'Never jump over a lazy dog.',
        'A brown fox is quick and agile.',
        'The dog is lazy and sleeps all day.',
        'The cat chases the mouse.'
    ]
}
df = pd.DataFrame(data)

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess_text(text):
    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to create the 'processed_document' column
df['processed_document'] = df['document'].apply(preprocess_text)

print("Original DataFrame:")
display(df[['document']])
print("\nDataFrame with Processed Document column:")
display(df[['processed_document']])

Original DataFrame:


,document
0,The quick brown fox jumps over the lazy dog.
1,Never jump over a lazy dog.
2,A brown fox is quick and agile.
3,The dog is lazy and sleeps all day.
4,The cat chases the mouse.



DataFrame with Processed Document column:


,processed_document
0,quick brown fox jump lazy dog
1,never jump lazy dog
2,brown fox quick agile
3,dog lazy sleep day
4,cat chase mouse


In [11]:
vectorizer = TfidfVectorizer(max_features=1000) # You can adjust max_features based on your dataset size
tfidf_matrix = vectorizer.fit_transform(df['processed_document'])

print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

# Display the vocabulary learned by the vectorizer
print("\nVocabulary (top 10 terms):", vectorizer.get_feature_names_out()[:10])

TF-IDF Matrix Shape: (5, 13)

Vocabulary (top 10 terms): ['agile' 'brown' 'cat' 'chase' 'day' 'dog' 'fox' 'jump' 'lazy' 'mouse']


In [12]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower() # Lowercasing
    text = re.sub(r'[^a-z\s]', '', text) # Remove punctuation and numbers
    tokens = word_tokenize(text) # Tokenization
    tokens = [word for word in tokens if word not in stop_words] # Stop word removal
    tokens = [lemmatizer.lemmatize(word) for word in tokens] # Lemmatization
    return ' '.join(tokens)

df['processed_document'] = df['document'].apply(preprocess_text)

print("DataFrame after preprocessing:")
display(df)

DataFrame after preprocessing:


,document,processed_document
0,The quick brown fox jumps over the lazy dog.,quick brown fox jump lazy dog
1,Never jump over a lazy dog.,never jump lazy dog
2,A brown fox is quick and agile.,brown fox quick agile
3,The dog is lazy and sleeps all day.,dog lazy sleep day
4,The cat chases the mouse.,cat chase mouse


In [13]:
vectorizer = TfidfVectorizer(max_features=1000) # You can adjust max_features based on your dataset size
tfidf_matrix = vectorizer.fit_transform(df['processed_document'])

print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

# Display the vocabulary learned by the vectorizer
print("\nVocabulary (top 10 terms):", vectorizer.get_feature_names_out()[:10])

TF-IDF Matrix Shape: (5, 13)

Vocabulary (top 10 terms): ['agile' 'brown' 'cat' 'chase' 'day' 'dog' 'fox' 'jump' 'lazy' 'mouse']


In [14]:
n_components = 2 # Let's extract 2 latent topics for this small dataset
lsa_model = TruncatedSVD(n_components=n_components, random_state=42)
lsa_matrix = lsa_model.fit_transform(tfidf_matrix)

print("LSA Matrix (Document-Topic Matrix) Shape:", lsa_matrix.shape)
print("\nLSA Matrix (first 5 documents, 2 topics):\n", lsa_matrix[:5])

# Get the components (Term-Topic Matrix)
components = lsa_model.components_
print("\nComponents (Term-Topic Matrix) Shape:", components.shape)

LSA Matrix (Document-Topic Matrix) Shape: (5, 2)

LSA Matrix (first 5 documents, 2 topics):
 [[ 9.19013421e-01  2.10817716e-01]
 [ 6.88754298e-01 -4.84187364e-01]
 [ 5.94033957e-01  7.43552853e-01]
 [ 5.17500486e-01 -5.83485106e-01]
 [ 3.50939988e-17  6.52630526e-17]]

Components (Term-Topic Matrix) Shape: (2, 13)


In [15]:
terms = vectorizer.get_feature_names_out()

for i, topic in enumerate(components):
    top_terms_idx = topic.argsort()[-5:][::-1] # Get indices of top 5 terms
    top_terms = [terms[idx] for idx in top_terms_idx]
    print(f"Topic {i+1}: {', '.join(top_terms)}")

print("\nDocument-Topic distribution (first document):")
print(lsa_matrix[0])

Topic 1: lazy, dog, jump, quick, fox
Topic 2: quick, fox, brown, agile, cat

Document-Topic distribution (first document):
[0.91901342 0.21081772]
